In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
from statsmodels.stats import multitest as smm
import holoviews as hv
import hvplot.pandas
import panel as pn
import random 
from functools import partial
hv.extension("bokeh")

In [ ]:
np.random.seed(710319)
n_1 = 30
n_2 = 30
ninst = 5000
n_scales = 20
ALPHA = 0.05

In [ ]:
ks_test_wmethod = np.vectorize(
    stats.mstats.ks_2samp, signature="(n),(n)->(),()", excluded=["method"]
)
ks_test_nomethod = np.vectorize(stats.mstats.ks_2samp, signature="(n),(n)->(),()")

In [ ]:
min_diff = 1
max_diff = 8
n_diffs = 20
ninst = 1000
ntotens = 120

pvals_ens = []
scales = np.linspace(min_diff, max_diff, n_diffs)
rvs1 = []
rvs2 = []

rvs1 = np.array([[stats.norm.rvs(size=ntotens, loc=0, scale=1) for _ in scales] for _ in range(ninst)])
rvs2 = np.array([[stats.norm.rvs(size=ntotens, loc=0, scale=_scale) for _scale in scales] for _ in range(ninst)])

In [ ]:
def explore(rvs, alpha=0.05, n_loc_ens=30):  
    rvs1, rvs2 = rvs
    pvals_ens = []
    _smp = random.sample(list(range(rvs1.shape[-1])), n_loc_ens)
    
    for _xi in range(rvs1.shape[1]):
        _, p_val = ks_test_nomethod(rvs1[:, _xi, _smp], rvs2[:, _xi, _smp])
        pvals_ens.append(p_val)

    pvals_ens = np.array(pvals_ens)
    pvals_all = {"Uncorrected": pvals_ens}

    _methods = [
        "fdr_bh",
        "fdr_by",
        "fdr_tsbh",
        "fdr_tsbky",
    ]
    for _method in _methods:
        _pvals = np.array(
            [
                smm.multipletests(
                pvals=pvals_ens[:, iix],
                alpha=0.01,
                method=_method,
                is_sorted=False,
                )[1]
                for iix in range(pvals_ens.shape[1])
            ]
        ).T
    
        pvals_all[_method] = _pvals

    rejected = {_method: (pvals_all[_method] < ALPHA).sum(axis=1) for _method in pvals_all}
    rejected["scales"] = scales - 1

    return pd.DataFrame(rejected)

In [ ]:
_exp = partial(explore, (rvs1, rvs2))
nens = pn.widgets.IntSlider(value=30, start=2, end=ntotens, step=2, name=f"NEns (of {ntotens})")
alpha = pn.widgets.FloatSlider(value=0.05, start=0.01, end=0.1, step=0.001, name="Alpha")
reject_dfi = hvplot.bind(_exp, alpha, nens).interactive().hvplot(x="scales", grid=True, line_width=5, alpha=0.6)

In [ ]:
reject_dfi.opts(width=900, height=600)